In [ ]:
import pandas as pd
import sys, os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from crimebb import *

In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas()

In [ ]:
YEAR="2021"

In [ ]:
DATA_PATH="../../data/"
CSV_PATH = f"{DATA_PATH}csv/{YEAR}/summary/"
CSV_PROCESSED = f"{DATA_PATH}csv/{YEAR}/processed/"

In [ ]:
verifyDir(CSV_PROCESSED)

### Loading data

In [ ]:
crimeBB_data = CrimeBBManager(DATA_PATH, YEAR)

#### Members

In [ ]:
crimeBB_data.process_members()

#### Boards

In [ ]:
crimeBB_data.process_boards()

#### Website

In [ ]:
crimeBB_data.process_sites()

#### Threads

In [ ]:
crimeBB_data.process_threads()

#### Posts

In [ ]:
crimeBB_data.process_posts()

### Summarizing CrimeBB

In [ ]:
website_df = pd.read_csv(f"{CSV_PROCESSED}sites.csv", sep="\t", low_memory=False)
members_df = pd.read_csv(f"{CSV_PROCESSED}members.csv", sep="\t", low_memory=False)
boards_df = pd.read_csv(f"{CSV_PROCESSED}boards.csv", sep="\t", low_memory=False)
threads_df = pd.read_csv(f"{CSV_PROCESSED}threads.csv", sep="\t", low_memory=False)

In [ ]:
chunk_size = 1000000

posts_reader = pd.read_csv(f"{CSV_PROCESSED}posts.csv", sep="\t", low_memory=False, iterator=True)
            
posts_df = pd.DataFrame()

len_readed=chunk_size
while len_readed>=chunk_size:
    current_df = posts_reader.get_chunk(chunk_size).copy()
    current_df.drop_duplicates(inplace=True)

    posts_df = pd.concat([posts_df, current_df], ignore_index=True)

    len_readed = current_df.shape[0]

In [ ]:
posts_df.info()

#### Merging

In [ ]:
posts_threads_df = pd.merge(posts_df, threads_df[["site_id", "board_id", "thread_id", "thread_title"]].drop_duplicates(), on=["site_id", "board_id", "thread_id"], how="left")

In [ ]:
posts_threads_boards_df = pd.merge(posts_threads_df, boards_df[["site_id", "site_name", "board_id", "board_title"]].drop_duplicates(), on=["site_id", "board_id"], how="left")

In [ ]:
crimebb_df = posts_threads_boards_df[['post_id', 'site_id', 'board_id', 'thread_id', 'user_id', 
                                     'site_name', 'board_title', 'thread_title', 'username', 'content', 
                                     'user_reputation', 'post_data_creation']].copy()

In [ ]:
crimebb_df.isnull().any()

In [ ]:
crimebb_df

In [ ]:
crimebb_df.to_csv(f"{CSV_PROCESSED}crimeBB_{YEAR}.csv", sep='\t', index=False)

### Drawing Missing Values

#### user_id equals None

In [ ]:
none_members_users = members_df[ (members_df["username"].str.lower()=="none") | (members_df["username"].isna()) ].copy().drop_duplicates()

In [ ]:
none_threads_users = threads_df[ (threads_df["username"].str.lower()=="none") | (threads_df["username"].isna()) ].copy().drop_duplicates()

In [ ]:
none_posts_users = posts_df[ (posts_df["username"].str.lower()=="none") | (posts_df["username"].isna()) ].copy().drop_duplicates()

#### thread title None

In [ ]:
none_threads_title = threads_df[threads_df["thread_title"].isna()]

In [ ]:
none_posts_threads_title = posts_threads_df[posts_threads_df["thread_title"].isna()]

#### posts content None

In [ ]:
none_posts_content = posts_threads_df[posts_threads_df["content"].isna()]

#### Draw

In [ ]:
import matplotlib.pyplot as plt

# None usernames

legends_users = ["members", "threads", "posts"]
values_users = [len(none_members_users), len(none_threads_users), len(none_posts_users)]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(legends_users,values_users, label='Null Usernames')
for x,y,p in zip(legends_users, values_users, values_users):
  plt.text(x, y, p)

# None threads title

legends_threads = ["threads ", "posts "]
values_threads = [len(none_threads_title), len(none_posts_threads_title)]

ax.bar(legends_threads, values_threads, label='Null Thread title')
for x,y,p in zip(legends_threads, values_threads, values_threads):
  plt.text(x, y, p)

# None posts content

legends_posts = ["posts  "]
values_posts = [len(none_posts_content)]

ax.bar(legends_posts, [len(none_posts_content)], label='Null Post content')
for x,y,p in zip(legends_posts, values_posts, values_posts):
  plt.text(x, y, p)

ax.legend()
ax.set_title("Missing values per table")
ax.set_ylabel("Number of missing values")
plt.show()